In [1]:
import snap
from os import path

In [2]:
BASE_PATH = "../data/academia.stackexchange.com"
# FOLDED_USERID_TSV_PATH = path.join(BASE_PATH, "userid_folded_graph.tsv")
FOLDED_USERID_GRAPH_PATH = path.join(BASE_PATH, "Userid_Folded_Graph.graph")
# FOLDED_POSTID_TSV_PATH = path.join(BASE_PATH, "postid_folded_graph.tsv")
COMMUNITIES_PATH = path.join(BASE_PATH, 'communities.txt')
COMMUNITIES_VEC_PATH = path.join(BASE_PATH, 'communities.vector')

In [3]:
# user_graph = snap.LoadEdgeList(snap.PUNGraph, FOLDED_USERID_TSV_PATH, 0, 1)
# user_graph.GetNodes()
# user_graph.GetEdges()

In [4]:
f_in = snap.TFIn(FOLDED_USERID_GRAPH_PATH)
user_graph = snap.TUNGraph.Load(f_in)
print "nodes", user_graph.GetNodes()
print "edges", user_graph.GetEdges()

nodes 18653
edges 137062264


In [7]:
# CNM
assert snap.CntSelfEdges(user_graph) == 0

comm_vec = snap.TCnComV()
modularity = snap.CommunityCNM(user_graph, comm_vec)

f_out = snap.TFOut(COMMUNITIES_VEC_PATH)
comm_vec.Save(f_out)
f_out.Flush()

In [8]:
f_in = snap.TFIn(COMMUNITIES_VEC_PATH)
comm_vec = snap.TCnComV()
comm_vec.Load(f_in)

print "communities", len(comm_vec)

with open(COMMUNITIES_PATH, 'w') as f:
    for i, comm in enumerate(comm_vec):
        f.write("#####Community {}#####\n".format(i))
        for node in comm:
            f.write(str(node) + '\n')
    f.write("The modularity of the network is {}".format(modularity))

communities 2


In [9]:
# modularity of communities
f_in = snap.TFIn(COMMUNITIES_VEC_PATH)
comm_vec = snap.TCnComV()
comm_vec.Load(f_in)

for i, comm in enumerate(comm_vec):
    community = snap.TIntV()
    for node in comm:
        community.Add(node)
    print 'Community {} modularity: {}'.format(i, snap.GetModularity(user_graph, community, user_graph.GetEdges()))

Community 0 modularity: 0.0104364555711
Community 1 modularity: 0.0104364555711
